# Dataset exploration

### CoPhIR dataset
- composed of multiple extracted descriptors (i.e., vectors) from images (e.g. color histogram)
- more info: https://ieeexplore.ieee.org/document/5271953

In [1]:
%load_ext autoreload
%autoreload 2

CoPhIR's original size is 100M, we worked with 1M version, but we potentially want to go to 5M with the next paper. I have saved the 1M and 100k versions here:

In [2]:
from enum import Enum
class DatasetDirs(Enum):
    COPHIR_1M = "/storage/brno6/home/tslaninakova/learned-indexes/MTree1M"
    COPHIR_100k = "/storage/brno6/home/tslaninakova/learned-indexes/MTree100k"
    PROFI_1M = "/storage/brno6/home/tslaninakova/learned-indexes/MtreeProfi2000"
    
class DatasetDirsLocal(Enum):
    COPHIR_100k = "./Mtree-Cophir-100k"

- other variants (if necessary) found in `cybela14@fi.muni.cz:/home/xdohnal/research/learning-indexes`, you can ssh to cybela14 from aisa

Use the local variant, if you don't have an access to metacentrum yet

In [2]:
%ls $DatasetDirs.COPHIR_100k.value

knn_gt.json  level-2.txt  objects.txt


### Directory description:
- `objects.txt` - the data
- `level-2.txt` - the assigned node labels from an M-tree = the 'ground truth' used for supervised learning
- `knn_gt.json` - dumped dictionary of 30 nearest objects to every object of the 1k subset we used for experiments

## Raw data
First three rows describe the data, the rest are the numeric descriptors (i.e., the data)

In [4]:
!head -n 9 $DatasetDirs.COPHIR_100k.value$'/objects.txt'

#objectKey messif.objects.keys.BucketIdObjectKey 1264121
#filter messif.objects.PrecomputedDistancesFixedArrayFilter 
ColorLayoutType;messif.objects.impl.ObjectColorLayout;ColorStructureType;messif.objects.impl.ObjectShortVectorL1;EdgeHistogramType;messif.objects.impl.ObjectVectorEdgecomp;HomogeneousTextureType;messif.objects.impl.ObjectHomogeneousTexture;ScalableColorType;messif.objects.impl.ObjectIntVectorL1
8,17,20,11,17,9;18,16,13;46,15,17
38,5,0,0,0,0,0,9,255,62,1,18,0,0,0,0,0,0,0,0,0,0,15,6,254,109,5,27,0,0,0,0,0,0,0,0,125,25,0,0,135,0,0,12,12,0,0,4,4,0,0,6,88,0,0,1,71,0,0,0,0,0,0,13
4,3,2,1,1,0,2,4,3,0,4,1,0,4,0,6,0,0,0,2,3,2,0,0,0,5,3,7,7,1,5,2,1,3,1,6,0,0,1,0,2,4,4,0,1,2,4,6,4,5,2,4,3,5,6,3,1,3,3,3,3,2,2,7,4,4,3,6,3,5,3,2,5,5,7,4,1,2,1,4
45;51;166,146,151,187,158,144,168,136,127,162,130,136,140,100,83,143,89,97,106,60,53,110,60,47,96,17,9,65,11,7;166,142,147,188,158,139,162,138,122,164,120,132,135,92,71,138,77,91,107,48,50,106,54,44,106,9,3,66,4,4
-78,-41,-62,79,53,14,22,39,-3

I load them and combine the numeric values into 1 row:

### Profiset dataset

In [ ]:
from LMI import LMI
li = LMI(DatasetDirs.PROFI_1M.value)
df = li.get_dataset()
df.head(2)

Loading labels


/auto/brno6/home/wollf/learned-indexes/learned-indexes/dataset_parsing.py:147: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_i = pd.concat([df_i, df_])


Loading descriptors


In [5]:
import numpy as np
df["L2"] = df["L2"].astype(np.int64)
df.head()

NameError: name 'df' is not defined

In [6]:
df.info()

NameError: name 'df' is not defined

### CoPhIR dataset

In [25]:
from LMI import LMI
li = LMI(DatasetDirs.COPHIR_100k.value)
df = li.get_dataset(normalize=False)
df.head(2)

24-02-21 19:15 INFO: Loaded dataset of shape: (100000, 285)


array([ 8,  8,  8, ..., 10, 10, 10])

L1, L2 are labels from `level-2.txt` indicating that the original M-tree had a depth of 2.

In [39]:
import pandas as pd
sorted(pd.unique(df["L2"].values))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100]

## Training a simple unsupervised model on a subset of the data

In [4]:
df_s = df.sample(frac=0.01)
training_specs = {"GMM": [{'comp': df_s["L1"].max(), 'cov_type': 'diag'}, {'comp': df_s["L2"].max(), 'cov_type': 'diag'}]}

training_specs = {"Faiss": [{'buckets': df_s["L1"].max()}, {'buckets': df_s["L2"].max()}], {}]}

%time df_result = li.train(df_s, training_specs, should_erase=True)
df_result.shape

24-02-21 19:00 INFO: Training GMM model with values of shape (1000, 282): n. of clusters=15 | covariance type=diag
24-02-21 19:01 INFO: Training level 1
24-02-21 19:01 INFO: Training GMM model with values of shape (29, 282): n. of clusters=100 | covariance type=diag
24-02-21 19:01 WARNING: Reducing the number of components from 100 to 14 since the number of                           training samples (29) is less than 100
24-02-21 19:01 INFO: Training GMM model with values of shape (43, 282): n. of clusters=14 | covariance type=diag
24-02-21 19:01 INFO: Training GMM model with values of shape (33, 282): n. of clusters=14 | covariance type=diag
24-02-21 19:01 INFO: Training GMM model with values of shape (55, 282): n. of clusters=14 | covariance type=diag
24-02-21 19:01 INFO: Training GMM model with values of shape (16, 282): n. of clusters=14 | covariance type=diag
24-02-21 19:01 INFO: Training GMM model with values of shape (100, 282): n. of clusters=14 | covariance type=diag
24-02-21 

CPU times: user 651 ms, sys: 7.17 ms, total: 659 ms
Wall time: 682 ms


(1000, 287)

In [18]:
df.shape

(100000, 285)

In [40]:
df.iloc[0]

L1                 8
L2                31
object_id    1264121
0                  8
1                 17
              ...   
277               -2
278                1
279                0
280               -3
281               -3
Name: 0, Length: 285, dtype: int64

In [41]:
from sklearn.cluster import KMeans

In [45]:
X = df.head(10)
kmeans = KMeans(n_clusters = 2).fit(X.drop(["L1", "L2", "object_id"], axis = 1))

In [50]:
df.iloc[11]

L1                 8
L2                31
object_id    2507363
0                 21
1                 18
              ...   
277               -3
278                8
279                0
280               -3
281               -3
Name: 11, Length: 285, dtype: int64

In [58]:
kmeans.transform([df.iloc[11].drop(["L1", "L2", "object_id"])])[0]

array([648.19715794, 702.85204702])

In [60]:
df.iloc[11]

L1                 8
L2                31
object_id    2507363
0                 21
1                 18
              ...   
277               -3
278                8
279                0
280               -3
281               -3
Name: 11, Length: 285, dtype: int64

In [55]:
[df.iloc[11].drop(["L1", "L2", "object_id"])]

[0      21
 1      18
 2       8
 3       6
 4      14
        ..
 277    -3
 278     8
 279     0
 280    -3
 281    -3
 Name: 11, Length: 282, dtype: int64]

In [24]:
df["L2"].max()

100

In [22]:
df_result.shape

(1000, 287)

In [7]:
result = li.search(df_result, df_result.iloc[0]["object_id"], stop_cond_objects=[500], debug=True)

07-01-21 08:21 INFO: Step 1: L1 added - PQ: [{'M.1.10': 1.0}, {'M.1.13': 1.1329272087579044e-78}, {'M.1.3': 3.4040043314400125e-96}, {'M.1.4': 3.016297205302193e-117}, {'M.1.5': 4.121877498142086e-133}, {'M.1.1': 2.6509091053163416e-142}, {'M.1.12': 4.3586168254717865e-150}, {'M.1.2': 3.529833008600152e-192}, {'M.1.7': 6.895773880000384e-204}, {'M.1.14': 7.679956954122186e-223}, {'M.1.6': 2.8635390938856102e-263}, {'M.1.0': 0.0}, {'M.1.8': 0.0}, {'M.1.9': 0.0}, {'M.1.11': 0.0}]

07-01-21 08:21 INFO: Popped M.1.10
07-01-21 08:21 INFO: L2 added - PQ (Top 5): [{'C.1.10.11': 1.0}, {'M.1.13': 1.1329272087579044e-78}, {'M.1.3': 3.4040043314400125e-96}, {'M.1.4': 3.016297205302193e-117}, {'M.1.5': 4.121877498142086e-133}]

07-01-21 08:21 INFO: L2 found bucket C.1.10.11
07-01-21 08:21 INFO: Popped M.1.13
07-01-21 08:21 INFO: L2 added - PQ (Top 5): [{'C.1.13.17': 1.0}, {'M.1.3': 3.4040043314400125e-96}, {'M.1.4': 3.016297205302193e-117}, {'C.1.13.10': 2.9856893995487625e-122}, {'M.1.5': 4.12187

In [8]:
result

{'id': 100846880,
 'time_checkpoints': [0.0854487419128418],
 'popped_nodes_checkpoints': [['M.1.10',
   'C.1.10.11',
   'M.1.13',
   'C.1.13.17',
   'M.1.3',
   'C.1.3.1',
   'C.1.3.8',
   'C.1.3.6',
   'M.1.4',
   'C.1.4.4',
   'C.1.4.8',
   'C.1.13.10',
   'M.1.5',
   'C.1.5.17',
   'C.1.5.1',
   'C.1.5.8',
   'C.1.5.3',
   'C.1.5.14',
   'C.1.5.20',
   'C.1.5.16',
   'C.1.5.18',
   'C.1.5.5',
   'M.1.1',
   'C.1.1.1',
   'C.1.4.12',
   'M.1.12',
   'C.1.12.3',
   'C.1.5.6',
   'C.1.4.3',
   'M.1.2',
   'C.1.2.1',
   'C.1.5.0',
   'M.1.7',
   'C.1.7.0',
   'M.1.14',
   'C.1.14.1',
   'C.1.5.13',
   'C.1.3.7',
   'M.1.6',
   'C.1.6.5',
   'C.1.10.15',
   'C.1.3.11',
   'C.1.10.7',
   'M.1.0',
   'C.1.0.4',
   'M.1.8',
   'C.1.8.2',
   'M.1.9',
   'C.1.9.7',
   'M.1.11',
   'C.1.11.3',
   'C.1.10.0',
   'C.1.10.1',
   'C.1.10.2',
   'C.1.10.3',
   'C.1.10.4',
   'C.1.10.5',
   'C.1.10.6',
   'C.1.10.8',
   'C.1.10.9',
   'C.1.10.10',
   'C.1.10.12',
   'C.1.10.13',
   'C.1.10.14',
   

`popped_nodes_checkpoints` gives us a list of buckets that were visited. k-NN performance is then calculated by checking the 30 nearest neighbors to the query searched:

## Evaluating the k-NN performance
- get the 30-NN objects for a query
- find out their buckets within the trained LMI
- make an intersection with the visited buckets in `popped_nodes_checkpoints`

In [19]:
# if not in metacentrum yet, download from https://www.fi.muni.cz/~xslanin/learned-indexes/cophir-knn-gt.json
knn_file = "/storage/brno6/home/tslaninakova/learned-indexes/cophir-1M-ground-truth-30NN.json"
knns = li.get_knn_ground_truth(filename=knn_file)

In [21]:
knns[str(df_result.iloc[0]["object_id"])]

{'365390735': 0.62756324,
 '221303952': 0.66440886,
 '439487761': 0.6693677,
 '413128212': 0.59656847,
 '361195928': 0.70481867,
 '169004575': 0.68627876,
 '100846880': 0.0,
 '167871401': 0.7212031,
 '112634416': 0.65926826,
 '270394545': 0.64337313,
 '429564726': 0.7034012,
 '378128695': 0.7249266,
 '100846905': 0.71257246,
 '233966395': 0.69729507,
 '100846652': 0.6643024,
 '398143810': 0.67309815,
 '169743263': 0.6781807,
 '433135174': 0.72369945,
 '152908623': 0.6666877,
 '374421076': 0.68043226,
 '4807766': 0.70307934,
 '171683168': 0.7044696,
 '102575842': 0.7082357,
 '109913828': 0.6794179,
 '275449829': 0.65825945,
 '398143676': 0.5878138,
 '350302872': 0.7166533,
 '250630251': 0.6780022,
 '248322158': 0.5889049,
 '100846845': 0.52966833}

These are the 30 nearest objects of the searched query. Since we only trained on a subset of the dataset, most of them are not in the trained portion (most likely only the query itself is), so we can't properly evaluate 30-NN performance. For that we'd have to train on the entire 1M dataset.

In [27]:
for key in knns[str(df_result.iloc[0]["object_id"])].keys():
    if df_result[df_result["object_id"] == int(key)].shape[0] == 0:
        print(f"{key} is not in df_result")
    else:
        print(f"{key} is in df_result, its buckets: {df_result[df_result['object_id'] == int(key)][['L1_pred', 'L2_pred']].values}")

365390735 is not in df_result
221303952 is not in df_result
439487761 is not in df_result
413128212 is not in df_result
361195928 is not in df_result
169004575 is not in df_result
100846880 is in df_result, its buckets: [[10 11]]
167871401 is not in df_result
112634416 is not in df_result
270394545 is not in df_result
429564726 is not in df_result
378128695 is not in df_result
100846905 is not in df_result
233966395 is not in df_result
100846652 is not in df_result
398143810 is not in df_result
169743263 is not in df_result
433135174 is not in df_result
152908623 is not in df_result
374421076 is not in df_result
4807766 is not in df_result
171683168 is not in df_result
102575842 is not in df_result
109913828 is not in df_result
275449829 is not in df_result
398143676 is not in df_result
350302872 is not in df_result
250630251 is not in df_result
248322158 is not in df_result
100846845 is not in df_result
